In [3]:
import numpy as np
import pandas as pd

from skrough.dataprep import (
    prepare_factorized_array,
    prepare_factorized_data,
    prepare_factorized_vector,
)

In [3]:
# df_train = pd.read_csv("../../1m_sdiv_balanced.csv", index_col=0)
# target_train = pd.read_csv("../../1m_sdiv_y.csv", index_col=0, header=None).reset_index(drop=True)[1]

df_train = pd.read_csv(
    "../../26m_sdiv_biased.csv", index_col=0, dtype=np.int8, nrows=1000000
)
target_train = pd.read_csv(
    "../../26m_sdiv_y.csv", index_col=0, header=None, dtype=np.int8, nrows=1000000
).reset_index(drop=True)[1]

In [4]:
df = pd.DataFrame(
    np.array(
        [
            ["sunny", "hot", "high", "weak", "no"],
            ["sunny", "hot", "high", "strong", "no"],
            ["overcast", "hot", "high", "weak", "yes"],
            ["rain", "mild", "high", "weak", "yes"],
            ["rain", "cool", "normal", "weak", "yes"],
            ["rain", "cool", "normal", "strong", "no"],
            ["overcast", "cool", "normal", "strong", "yes"],
            ["sunny", "mild", "high", "weak", "no"],
            ["sunny", "cool", "normal", "weak", "yes"],
            ["rain", "mild", "normal", "weak", "yes"],
            ["sunny", "mild", "normal", "strong", "yes"],
            ["overcast", "mild", "high", "strong", "yes"],
            ["overcast", "hot", "normal", "weak", "yes"],
            ["rain", "mild", "high", "strong", "no"],
        ],
        dtype=object,
    ),
    columns=["Outlook", "Temperature", "Humidity", "Wind", "Play"],
)
TARGET_COLUMN = "Play"
x, x_counts, y, y_count = prepare_factorized_data(df, TARGET_COLUMN)

In [4]:
x, x_counts = prepare_factorized_array(df_train.to_numpy())
y, y_count = prepare_factorized_vector(target_train.to_numpy())

# x, x_counts = prepare_factorized_array(df.to_numpy())
# y, y_count = prepare_factorized_vector(target)

In [448]:
import numpy as np

from skrough.algorithms import hooks
from skrough.algorithms.key_names import (
    CONFIG_CANDIDATES_SELECT_RANDOM_MAX_COUNT,
    CONFIG_CHAOS_FUN,
    CONFIG_CONSECUTIVE_EMPTY_ITERATIONS_MAX_COUNT,
    CONFIG_DAAR_ALLOWED_RANDOMNESS,
    CONFIG_DAAR_N_OF_PROBES,
    CONFIG_EPSILON,
    CONFIG_SELECT_ATTRS_CHAOS_SCORE_BASED_MAX_COUNT,
    INPUT_DATA_X,
    INPUT_DATA_Y,
    VALUES_CHAOS_SCORE_APPROX_THRESHOLD,
    VALUES_CHAOS_SCORE_BASE,
    VALUES_CHAOS_SCORE_TOTAL,
    VALUES_GROUP_INDEX,
    VALUES_RESULT_ATTRS,
    VALUES_RESULT_OBJS,
    VALUES_X,
    VALUES_X_COUNTS,
    VALUES_Y,
    VALUES_Y_COUNT,
)
from skrough.algorithms.meta import processing, stage
from skrough.algorithms.reusables.daar import daar_stage
from skrough.chaos_measures import entropy, gini_impurity
from skrough.chaos_score import get_chaos_score_stats
from skrough.structs.state import ProcessingState

msfun = processing.ProcessingMultiStage.from_hooks(
    init_multi_stage_hooks=[
        hooks.init_hooks.init_hook_factorize_data_x_y,
        hooks.init_hooks.init_hook_approx_threshold,
        hooks.init_hooks.init_hook_result_attrs_empty,
        hooks.init_hooks.init_hook_single_group_index,
    ],
    stages=[daar_stage],
    finalize_hooks=None,
    prepare_result_fun=hooks.prepare_result_hooks.prepare_result_hook_attrs_subset,
)


def compute():
    return msfun(
        input_data={
            INPUT_DATA_X: x,
            INPUT_DATA_Y: y,
        },
        config={
            CONFIG_CHAOS_FUN: entropy,
            CONFIG_EPSILON: 0.0,
            CONFIG_SELECT_ATTRS_CHAOS_SCORE_BASED_MAX_COUNT: 1,
            CONFIG_CANDIDATES_SELECT_RANDOM_MAX_COUNT: 1,
            CONFIG_DAAR_ALLOWED_RANDOMNESS: 0.9,
            CONFIG_DAAR_N_OF_PROBES: 100,
            CONFIG_CONSECUTIVE_EMPTY_ITERATIONS_MAX_COUNT: 1,
        },
    )


res = compute()
res

AttrsSubset(attrs=[1, 0, 2, 3])

In [8]:
res

AttrsSubset(attrs=[0, 3, 1])

In [47]:
# import joblib
# result = joblib.Parallel(n_jobs=4)(
#             joblib.delayed(compute)()
#             for _ in range(10)
#         )
# result

In [12]:
from skrough.checks import check_if_approx_reduct

In [20]:
check_if_approx_reduct(
    x, x_counts, y, y_count, chaos_fun=entropy, epsilon=0.5, attrs=[68]
)

True

In [10]:
res

AttrsSubset(attrs=[68, 69, 66, 58, 59, 57])